In [1]:
!pip install -U -q evaluate

In [2]:
import os

os.environ['DATASET']='aisuko/phishing-binary-classification'
# os.environ['ORIGINAL_MODEL']="nvidia/Hymba-1.5B-Base"
os.environ['ORIGINAL_MODEL']="google-bert/bert-base-uncased"
os.environ['FT_MODEL']='phishing-binary-classification-bert'

In [3]:
from datasets import load_dataset

ds = load_dataset(os.getenv('DATASET'))
ds

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/22.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/528006 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/66001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/66001 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'labels'],
        num_rows: 528006
    })
    validation: Dataset({
        features: ['url', 'labels'],
        num_rows: 66001
    })
    test: Dataset({
        features: ['url', 'labels'],
        num_rows: 66001
    })
})

In [4]:
from datasets import DatasetDict

pre_processed_ds_train_low=ds['train'].shuffle(seed=42).select(range(10000))
pre_processed_ds_test_low=ds['test'].shuffle(seed=42).select(range(5000))
pre_processed_ds_validate_low=ds['validation'].shuffle(seed=42).select(range(5000))

ds_low=DatasetDict({
    'train': pre_processed_ds_train_low,
    'test': pre_processed_ds_test_low,
    'validation': pre_processed_ds_validate_low,
})

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer=AutoTokenizer.from_pretrained(os.getenv('ORIGINAL_MODEL'))
tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

# Tokenized the data 

In [6]:
# define text preprocessing
def preprocess_func(examples):
    return tokenizer(examples['url'], truncation=True)

# apply to every row of data
tokenized_ds_low=ds_low.map(preprocess_func, batched=True)
tokenized_ds_low

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['url', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['url', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['url', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})

# Load the model

In [7]:
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}


model=AutoModelForSequenceClassification.from_pretrained(
    os.getenv('ORIGINAL_MODEL'),
    num_labels=2,
    id2label=id2label, 
    label2id=label2id
)
model.to("cuda")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


# Freeze the pre-trained weights

In [9]:
for name, param in model.base_model.named_parameters():
    param.requires_grad=False

In [10]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

# Padding function for training

In [11]:
from transformers import DataCollatorWithPadding

data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
# Original code from https://github.com/ShawhinT/YouTube-Blog/blob/main/LLMs/model-compression/0_train-teacher.ipynb
# Acknowledge the original author
# Adapte by Bowen

import evaluate
import numpy as np


accuracy=evaluate.load("accuracy")
auc_score=evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    predictions, labels=eval_pred
    probabilities=np.exp(predictions)/ np.exp(predictions).sum(-1, keepdims=True)
    positive_class_probs=probabilities[:,1]
    # compute auc
    auc=np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

# Training model

In [13]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir=os.getenv("FT_MODEL"),
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="tensorboard",
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_dataset["train"],    
    # eval_dataset=tokenized_dataset["test"],
    train_dataset=tokenized_ds_low["train"],
    eval_dataset=tokenized_ds_low["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.668100,0.619765,0.690000,0.885000
2,0.618500,0.581258,0.712000,0.897000
3,0.590700,0.547759,0.820000,0.900000
4,0.569300,0.526668,0.815000,0.908000
5,0.560800,0.519315,0.787000,0.910000
6,0.548600,0.516771,0.769000,0.915000
7,0.540900,0.503415,0.790000,0.916000
8,0.533800,0.501570,0.784000,0.918000
9,0.533100,0.494655,0.796000,0.919000
10,0.530800,0.487794,0.820000,0.919000


TrainOutput(global_step=12500, training_loss=0.569460732421875, metrics={'train_runtime': 415.1478, 'train_samples_per_second': 240.878, 'train_steps_per_second': 30.11, 'total_flos': 2854500061821120.0, 'train_loss': 0.569460732421875, 'epoch': 10.0})

# Applying Model to Validation Dataset

In [14]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_ds_low["validation"])

In [15]:
# extract the logits and labels from the predictions object
logits=predictions.predictions
labels=predictions.label_ids

metrics=compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.834, 'AUC': 0.926}


In [16]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [17]:
kwargs={
    'model_name': os.getenv('DATASET'),
    'finetuned_from': os.getenv('ORIGINAL_MODEL'),
    'tasks': 'Text-Generation',
    'dataset': os.getenv('DATASET')
}

tokenizer.push_to_hub(os.getenv('DATASET'))
trainer.push_to_hub('**kwargs')

README.md:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

events.out.tfevents.1733120842.96dc7100556f.23.0:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aisuko/phishing-binary-classification-bert/commit/ca6893a92b3c8611ef002fafa96f198698a3e7aa', commit_message='**kwargs', commit_description='', oid='ca6893a92b3c8611ef002fafa96f198698a3e7aa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/aisuko/phishing-binary-classification-bert', endpoint='https://huggingface.co', repo_type='model', repo_id='aisuko/phishing-binary-classification-bert'), pr_revision=None, pr_num=None)

# References
* https://www.kaggle.com/code/aisuko/fine-tune-openelm-270m-with-orpo

# Acknowledge
* https://github.com/ShawhinT/YouTube-Blog/blob/main/LLMs/model-compression/0_train-teacher.ipynb